In [1]:
import os
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from collections import defaultdict
import sys

In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
def clean_text(cmnt_text, clean_wiki_tokens = True):
    cmnt_text = cmnt_text.lower()
    #removing links
    cmnt_text = re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", "", cmnt_text)
    #removing IP addresses
    cmnt_text = re.sub(r"(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)){3}", "", cmnt_text)
    
    if clean_wiki_tokens:
        #removing images
        cmnt_text = re.sub(r"image:[a-zA-Z0-9]*\.jpg", " ", cmnt_text)
        cmnt_text = re.sub(r"image:[a-zA-Z0-9]*\.png", " ", cmnt_text)
        cmnt_text = re.sub(r"image:[a-zA-Z0-9]*\.gif", " ", cmnt_text)
        cmnt_text = re.sub(r"image:[a-zA-Z0-9]*\.bmp", " ", cmnt_text)
        
        #removing CSS
        cmnt_text = re.sub(r"#([A-Fa-f0-9]{6}|[A-Fa-f0-9]{3})", " ",cmnt_text)
        cmnt_text = re.sub(r"\{\|[^\}]*\|\}", " ", cmnt_text)
        
        #removing templates
        cmnt_text = re.sub(r"\[?\[user:.*\]", " ", cmnt_text)
        cmnt_text = re.sub(r"\[?\[wikipedia:.*\]", " ", cmnt_text)
        cmnt_text = re.sub(r"\[?\[special:.*\]", " ", cmnt_text)
        cmnt_text = re.sub(r"\[?\[category:.*\]", " ", cmnt_text)
        
    cmnt_text = re.sub(r"what's", "what is ", cmnt_text)
    cmnt_text = re.sub(r"\'s", " ", cmnt_text)
    cmnt_text = re.sub(r"\'ve", " have ", cmnt_text)
    cmnt_text = re.sub(r"can't", " cannot ", cmnt_text)
    cmnt_text = re.sub(r"n't", " not ", cmnt_text)
    cmnt_text = re.sub(r"i'm", " i am ", cmnt_text)
    cmnt_text = re.sub(r"\'m", " i am ", cmnt_text)
    cmnt_text = re.sub(r"\'re", " are ", cmnt_text)
    cmnt_text = re.sub(r"\'d", " would ", cmnt_text)
    cmnt_text = re.sub(r"\'ll", " will ", cmnt_text)
    cmnt_text = re.sub(r",", " ", cmnt_text)
    cmnt_text = re.sub(r"\.", " ", cmnt_text)
    cmnt_text = re.sub(r"!", " ! ", cmnt_text)
    cmnt_text = re.sub(r"\/", " ", cmnt_text)
    cmnt_text = re.sub(r"\?", " ? ", cmnt_text)
    cmnt_text = re.sub(r"\!", " ! ", cmnt_text)
    cmnt_text = re.sub(r"\"", " ", cmnt_text)
    cmnt_text = re.sub(r"\^", " ^ ", cmnt_text)
    cmnt_text = re.sub(r"\+", " + ", cmnt_text)
    cmnt_text = re.sub(r"\-", " - ", cmnt_text)
    cmnt_text = re.sub(r"\=", " = ", cmnt_text)
    cmnt_text = re.sub(r"'", " ", cmnt_text)
    cmnt_text = re.sub(r"(\d+)(k)", r"\g<1>000", cmnt_text)
    cmnt_text = re.sub(r":", " : ", cmnt_text)
    cmnt_text = re.sub(r" e g ", " eg ", cmnt_text)
    cmnt_text = re.sub(r" b g ", " bg ", cmnt_text)
    cmnt_text = re.sub(r" u s ", " american ", cmnt_text)
    cmnt_text = re.sub(r"\0s", "0", cmnt_text)
    cmnt_text = re.sub(r" 9 11 ", "911", cmnt_text)
    cmnt_text = re.sub(r"e - mail", "email", cmnt_text)
    cmnt_text = re.sub(r"j k", "jk", cmnt_text)
    cmnt_text = re.sub(r"\s{2,}", " ", cmnt_text)
    cmnt_text = re.sub(r"\n", " ", cmnt_text)
    
        
    return(cmnt_text)

In [4]:
train_df['comment_text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [5]:
txt = clean_text(train_df['comment_text'][0])

In [6]:
txt

'explanation why the edits made under my username hardcore metallica fan were reverted ? they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now '

In [7]:
train_df['clean_cmnts'] = train_df['comment_text'].apply(clean_text)

In [8]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_cmnts
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d aww ! he matches this background colour i am...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man i am really not trying to edit war it ...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,more i cannot make any real suggestions on im...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir are my hero any chance you remember wh...


In [9]:
train_df = train_df.drop(['comment_text'], axis=1)

In [10]:
train_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_cmnts
0,0000997932d777bf,0,0,0,0,0,0,explanation why the edits made under my userna...
1,000103f0d9cfb60f,0,0,0,0,0,0,d aww ! he matches this background colour i am...
2,000113f07ec002fd,0,0,0,0,0,0,hey man i am really not trying to edit war it ...
3,0001b41b1c6bb37e,0,0,0,0,0,0,more i cannot make any real suggestions on im...
4,0001d958c54c6e35,0,0,0,0,0,0,you sir are my hero any chance you remember wh...


In [11]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [12]:
X = train_df[['clean_cmnts']]
y = train_df[label_cols]

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score
import re, string

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [15]:
y_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
54568,0,0,0,0,0,0
3786,0,0,0,0,0,0
22938,0,0,0,0,0,0
137856,0,0,0,0,0,0
143038,0,0,0,0,0,0


In [16]:
X_train.head()

,clean_cmnts
54568,conflict of interest note by your user name i...
3786,(update : actually i changed this to something...
22938,mrca article sniperz11 thanks for your comment...
137856,arguing that bart and caltrain should get more...
143038,blocked hi i blocked you for 24 hours for bei...


In [17]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s):
    return re_tok.sub(r'\1', s).split()

In [47]:
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )

In [48]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((111699, 1), (47872, 1), (111699, 6), (47872, 6))

In [49]:
trn_vec_doc = vec.fit_transform(X_train['clean_cmnts'])
test_vec_doc = vec.transform(X_test['clean_cmnts'])

In [50]:
trn_vec_doc.shape, test_vec_doc.shape, y_train.shape, y_test.shape

((111699, 304899), (47872, 304899), (111699, 6), (47872, 6))

In [51]:
def model(y):
    lr=LogisticRegression(C=4, solver='saga')
    lr.fit(trn_vec_doc, y)
    return lr

In [52]:
preds = np.zeros((len(X_test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m = model(y_train[j])
    preds[:,i] = m.predict_proba(test_vec_doc)[:,1]

fit toxic
fit severe_toxic
fit obscene
fit threat
fit insult
fit identity_hate


In [53]:
p = preds.round()

for i, j in enumerate(label_cols):
    print(j)
    print(confusion_matrix(y_test[j], p[:, i]))
    print(f1_score(y_test[j], p[:, i]))
    print('\n')

toxic
[[43001   289]
 [ 1623  2959]]
0.7558109833971903


severe_toxic
[[47291    95]
 [  361   125]]
0.3541076487252125


obscene
[[45156   160]
 [  937  1619]]
0.7469434832756633


threat
[[47731     5]
 [  106    30]]
0.3508771929824561


insult
[[45213   270]
 [ 1098  1291]]
0.6536708860759494


identity_hate
[[47414    26]
 [  346    86]]
0.3161764705882353




In [39]:
p = preds.round()

for i, j in enumerate(label_cols):
    print(j)
    print(confusion_matrix(y_test[j], p[:, i]))
    print(f1_score(y_test[j], p[:, i]))
    print('\n')

toxic
[[42895   395]
 [ 1481  3101]]
0.7677642980935875


severe_toxic
[[47284   102]
 [  348   138]]
0.38016528925619836


obscene
[[45131   185]
 [  826  1730]]
0.7738760903600983


threat
[[47716    20]
 [  104    32]]
0.3404255319148936


insult
[[45173   310]
 [ 1046  1343]]
0.6645225136071251


identity_hate
[[47393    47]
 [  333    99]]
0.34256055363321797


